In [3]:
import h5py
import os

root = '/home/yigit/projects/mbcnp/data/raw/mocapact/'
files = []

# Iterate directory
for file_path in os.listdir(root):
    if file_path.endswith('.hdf5') and os.path.isfile(os.path.join(root, file_path)):
        # add filename to list
        files.append(file_path)
print(files)

import numpy as np

desired_observables = ['actuator_activation', 'joints_pos', 'joints_vel', 'sensors_gyro', 'end_effectors_pos', 
                       'sensors_torque', 'sensors_touch', 'sensors_velocimeter', 'world_zaxis']

def get_obs_indices(path):
    indices = []

    f = h5py.File(path, 'r+')
    walker_obs_dict = f['observable_indices']['walker']
    for k in walker_obs_dict.keys():
        if k in desired_observables:
            dum = walker_obs_dict[k][:]
            indices.extend(dum)
    f.close()

    return np.array(indices)

# Get indices
indices = get_obs_indices(os.path.join(root, 'CMU_049_06.hdf5'))

#region read mocapact data
full_obs, full_act = [], []

for file in files:
    fp = os.path.join(root, file)
    # Open file
    f = h5py.File(fp, 'r+')

    demos = {}

    num_start_rollouts = f['n_start_rollouts'][()]  # concatenate snippets to create this many rollouts
    for i in range(num_start_rollouts):
        demos.update({i: {}})
        demos[i].update({'obs': {}})
        demos[i].update({'act': {}})
    
    num_snippets = 0
    for key in f.keys():
        if key.startswith('CMU_'):
            num_snippets += 1

    for key in f.keys():
        if key.startswith('CMU_'):
            start, end = int(key.split('-')[-2]), int(key.split('-')[-1])
            for i in range(num_start_rollouts):
                obs = np.array(f[key][str(i)]['observations']['proprioceptive'])
                act = np.array(f[key][str(i)]['actions'])
                for j in range(len(act)):
                    demos[i]['obs'].update({start+j: obs[j, indices]})
                    demos[i]['act'].update({start+j: act[j]})

    for key in f.keys():
        for i in range(num_start_rollouts):
            if key.startswith('CMU_') and f[key]['early_termination'][i] == True:
                if i in demos.keys():
                    demos.pop(i)

    for key in demos.keys():
        full_obs.append(np.array(list(demos[key]['obs'].values())))
        full_act.append(np.array(list(demos[key]['act'].values())))

    f.close()

# print(len(full_obs), len(full_act))
#endregion
min_length = 1000
for i in range(len(full_obs)):
    if len(full_obs[i]) < min_length:
        min_length = len(full_obs[i])

processed_obs, processed_act = [], []
for i in range(len(full_obs)):
    processed_obs.append(full_obs[i][np.linspace(0, len(full_obs[i])-1, min_length, dtype=int)])
    processed_act.append(full_act[i][np.linspace(0, len(full_obs[i])-1, min_length, dtype=int)])



['CMU_049_06.hdf5', 'CMU_049_07.hdf5']


In [5]:
from models.wta_cnp import WTA_CNP
import torch

def get_available_gpu_with_most_memory():
    gpu_memory = []
    for i in range(torch.cuda.device_count()):
        torch.cuda.set_device(i)  # Switch to the GPU to accurately measure memory
        gpu_memory.append((i, torch.cuda.memory_stats()['reserved_bytes.all.current'] / (1024 ** 2)))

    gpu_memory.sort(key=lambda x: x[1], reverse=True)

    return gpu_memory[0][0]

if torch.cuda.is_available():
    available_gpu = get_available_gpu_with_most_memory()
    if available_gpu == 0:
        device = torch.device("cuda:0")
    else:
        device = torch.device(f"cuda:{available_gpu}")
else:
    device = torch.device("cpu")

print("Device :", device)

###

torch.set_float32_matmul_precision('high')

Device : cuda:0


In [9]:
batch_size = 2
n_max_obs, n_max_tar = 6, 6

t_steps = min_length
num_val = 4
num_demos = len(full_obs)-num_val
num_classes = 2
num_indiv = num_demos//num_classes  # number of demos per class

dx, dy = len(indices), len(full_act[0][0])

num_val_indiv = num_val//num_classes

colors = ['tomato', 'aqua']

x = torch.zeros(num_demos, t_steps, dx, device=device)
y = torch.zeros(num_demos, t_steps, dy, device=device)
vx = torch.zeros(num_val, t_steps, dx, device=device)
vy = torch.zeros(num_val, t_steps, dy, device=device)

ind = torch.randperm(len(full_obs))
vind = torch.cat((torch.randint(0, num_indiv, (num_val_indiv, 1)), torch.randint(num_indiv, num_demos, (num_val_indiv, 1))), dim=0)
tr_ctr, val_ctr = 0, 0

for i in range(len(full_obs)):
    if i in vind:
        vx[val_ctr] = torch.tensor(processed_obs[i], dtype=torch.float32)
        vy[val_ctr] = torch.tensor(processed_act[i], dtype=torch.float32)
        val_ctr += 1
    else:
        x[tr_ctr] = torch.tensor(processed_obs[i], dtype=torch.float32)
        y[tr_ctr] = torch.tensor(processed_act[i], dtype=torch.float32)
        tr_ctr += 1

print("X:", x.shape, "Y:", y.shape, "VX:", vx.shape, "VY:", vy.shape)

X: torch.Size([16, 122, 205]) Y: torch.Size([16, 122, 56]) VX: torch.Size([4, 122, 205]) VY: torch.Size([4, 122, 56])


In [10]:
import torch
from models.wta_cnp import WTA_CNP

root_folder = f'outputs/experimental/56D/1701871167/'
wta_model_path = f'{root_folder}saved_models/wta_on_synth.pt'

y = torch.load(f'{root_folder}y.pt').cpu()
num_samples, t_steps, dy = y.shape
dx = 205
batch_size = 1
n_max_obs, n_max_tar = 6, 6

wta = WTA_CNP(dx, dy, n_max_obs, n_max_tar, [1024, 1024, 1024], num_decoders=2, decoder_hidden_dims=[512, 512, 512], batch_size=batch_size, scale_coefs=True).to(device)

wta.load_state_dict(torch.load(wta_model_path))
wta.eval()

WTA_CNP(
  (encoder): Sequential(
    (0): Linear(in_features=261, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (decoders): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=1229, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=112, bias=True)
    )
  )
  (gate): Sequential(
    (0): Linear(in_features=1024, out_features=2, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [16]:
import os.path as osp
import numpy as np
import tree
import mujoco

from typing import Any, Callable, Dict, Optional, Text, Tuple
from dm_control import composer
from dm_control.locomotion.mocap import cmu_mocap_data
from dm_control.locomotion.mocap import loader
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import utils
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.walkers import initializers

class CartwheelInitializer(initializers.WalkerInitializer):
    def __init__(self):
        ref_path = cmu_mocap_data.get_path_for_cmu(version='2020')
        mocap_loader = loader.HDF5TrajectoryLoader(ref_path)
        trajectory = mocap_loader.get_trajectory('CMU_049_06')
        self._trajectory = trajectory
        clip_reference_features = trajectory.as_dict()
        clip_reference_features = tracking._strip_reference_prefix(clip_reference_features, 'walker/')
        self._cw_features = tree.map_structure(lambda x: x[0], clip_reference_features)

    def initialize_pose(self, physics, walker, random_state):
        del random_state
        utils.set_walker_from_features(physics, walker, self._cw_features)
        mujoco.mj_kinematics(physics.model.ptr, physics.data.ptr)

In [17]:
from dm_control import viewer
from dm_control.locomotion import arenas
from dm_control.locomotion.tasks import go_to_target


dind = 5

def prepare_obs(obs, ind, a):
    vv = []
    for k in obs.keys():
        real_key = k.split('/')[1]
        if real_key in desired_observables:
            vals = obs[k].flatten()
            vv.extend([vals])
    if ind == 0:
        vv.extend([y[dind, ind].numpy()])
    else:
        vv.extend([a])
    v = np.concatenate(vv).reshape(-1)
    return torch.from_numpy(v).view(1, 1, dx+dy).float().to(device)


def prepare_tar(ind):
    return x[dind, ind].view(1, 1, dx).float().to(device)


initializer = CartwheelInitializer()
walker = cmu_humanoid.CMUHumanoidPositionControlledV2020(initializer=initializer)
arena = arenas.Floor()

task = go_to_target.GoToTarget(walker=walker, arena=arena, physics_timestep=0.005, control_timestep=0.03)
env = composer.Environment(task=task, random_state=None)
# print(env.control_timestep())

initializer.initialize_pose(env.physics, walker, None)

ind = -1
inst_a = None
action_spec = env.action_spec()

def tst(ts):
    global ind, inst_a
    ind += 1
    dm_obs, dm_tar = prepare_obs(ts.observation, ind, inst_a), prepare_tar(ind+1)
    p_wta, g_wta = wta(dm_obs, dm_tar)
    # print(g_wta.squeeze(1))
    inst_a = p_wta[torch.argmax(g_wta.squeeze(1), dim=-1), 0, 0, :dy].cpu().detach().numpy().squeeze()
    return inst_a
    # return y[dind, ind].numpy()
    # print(env.physics.named.data.body_xpos['torso'].copy())
    # action = np.random.uniform(action_spec.minimum, action_spec.maximum, size=action_spec.shape)
    # return action

# Viewer for visualization
viewer.launch(env, policy=tst)

In [18]:
initializer._trajectory